## Tips Dataframe

- Loc:Desktop\Fundamentals_of-Data_Analysis/Fund_of_Data_Analysis/

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns; sns.set()
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':(10,8)})
sns.set_style('white')

### Description

In [ ]:
df=pd.read_csv("tips.csv")

In [2]:
print (('The data has the following shape'), df.shape)
print ("\n")
df.info()

NameError: name 'df' is not defined

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'total_bill': 'Bill', 'tip':'Tip', 'sex':'Gender', 'smoker':'Smoker', 'day':'Day','time':'Time', 'size': 'Party'})
df.head()

In [ ]:
df.describe()

In [ ]:
df.plot(style = "o", ms=3, figsize = [10,5])
plt.show()
#Ref1: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html
#Ref2:http://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/understand-df-plot-in-pandas/

In [ ]:
color_list = ['red' if i=='Male' else 'green' for i in df.loc[:,'Gender']]

pd.plotting.scatter_matrix(df, alpha=0.9, c = color_list, figsize = [20,20],
            diagonal = 'hist', s = 100, marker = '.')
plt.show()

In [ ]:
df.groupby(['Gender', 'Party'])['Tip'].mean().unstack(level=0).plot(kind='bar', figsize=(10,5))
plt.title('Tip amount by Party size and Gender')
plt.ylabel('Tip Amount')

In [ ]:
df['Tippc']=df.apply(lambda row: row.Tip / row.Bill, axis=1)
# Ref: https://kaijento.github.io/2017/04/22/pandas-create-new-column-sum/
# Tippc is the percent of the tip relative to the Bill

In [ ]:
df.head()

In [ ]:
df.groupby(['Gender', 'Time'])['Tippc'].mean().unstack(level=0).plot(kind='bar', figsize=(10,5))
plt.ylabel('Tip Amount as a percentage of the Bill')

In [ ]:
#df2 = df(columns=['Gender', 'Time'])
#df2 #.plot.bar()

### Linear Regression
- Plot the Tips relative to the Bill amount.

In [ ]:
df.plot(x='Bill', y='Tip', style='o')  
plt.title('Bill amount and Tip given')  
plt.xlabel('Bill')  
plt.ylabel('Tip')  
plt.show()

- Looking at where the dots, it would seem that the Tip amount increases relative to the Bill amount. It would seem that the Tip is approximate 20 % of the Bill amount. There are a few outliers that could skew the data.
- I will try a few values for the slope and where it crosses the Y-axis. The slopes I will at will vary between 0.1 and 0.2. (I did try a number of other values, but for this analysis I will use the outlined values). 

In [ ]:
# Plot w versus d with black dots.
df.plot(x='Bill', y='Tip', style='o')

# Overlay some lines on the plot.
x = np.arange(0.0, 50.0, 1.0)
plt.plot(x, 0.125 * x + 0.5, 'r-', label=r"$x/8 + 0.5$")
plt.plot(x, 0.2 * x +  0.5, 'g-', label=r"$x/5 +  0.5$")
plt.plot(x, 0.1 * x + 0.0, 'b-', label=r"$x/10 + 0.0$")

# Add a legend.
plt.legend()

# Add axis labels.
plt.xlabel('Bill')
plt.ylabel('Tips')
# Adjust the plot range
plt.ylim(0, 10)
# Show the plot.
plt.show()

- The red line seems to be the best fit. But I will investigate using the least square technique. In the above table, for every x, there is two values for y, one on the line and one on the dot. The following measures this distance and squares it (to account for negative values). The lease value is the best guess for the lines in the plot. 

In [ ]:
B = df["Bill"]
T = df["Tip"]

cost = lambda m,c: np.sum([(T[i] - m * B[i] - c)**2 for i in range(B.size)])

print("Red line cost with m = %5.2f and c = %5.2f: is %8.2f" % (0.125, 0.50, cost(0.125, 0.5)))
print("Green line cost with m = %5.2f and c = %5.2f: is %8.2f" % (0.2,  0.5, cost(0.2,  0.5)))
print("Blue line cost with m = %5.2f and c = %5.2f: is %8.2f" % (0.1, 0.0, cost(0.1, 0.0)))

### Minimising the cost

In [ ]:
# Calculate the best values for m and c.

B = df["Bill"]
T = df["Tip"]

# Calculate the best values for B and T.
B_avg = df["Bill"].mean()
T_avg = df["Tip"].mean()

B_zero = B - B_avg
T_zero = T - T_avg

m = np.sum(B_zero * T_zero) / np.sum(B_zero * B_zero)
c = T_avg - m * B_avg

print ("m is %8.6f and c is %6.6f." % (m, c))


x, y, x_avg, y_avg = B, T, B_avg, T_avg
m2 = (np.sum(x * y) - y_avg * np.sum(x)) / (np.sum(x * x) - x_avg * np.sum(x))
c2 = y_avg - m2 * x_avg
m2, c2

In [ ]:
np.polyfit(B, T, 1)

- The line the minimises the cost can now be superimposed on the data.

In [ ]:
# Plot the best fit line.
plt.plot(B, T, 'k.', label='Original data')
plt.plot(B, m * B + c, 'b-', label='Best fit line')

# Add axis labels and a legend.
plt.xlabel('Bill')
plt.ylabel('Tips')
plt.legend()

plt.ylim(0, 10)
# Show the plot.
plt.show()

In [ ]:
cost = lambda m,c: np.sum([(T[i] - m * B[i] - c)**2 for i in range(B.size)])

print("Red line cost with m = %5.2f and c = %5.2f: is %8.2f" % (0.105025, 0.920270, cost(0.11, 0.92)))

- In the case of linear regression, the cost function is the sum of the squares of the residuals (residuals being the difference between the dependent variable value and the value given by the model). The procedure finds for you the intercept and slope(s) that minimize the cost function.
- This is why the linear regression method is also called “Least-squares regression”.
- A line with slope =  0.11 and y-axia =  0.92: gives the lowest cost function, at 255.62.
- So this equation best fits the line.

In [3]:
import statsmodels.api as sm

X = df['Bill']
y = df['Tip']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

NameError: name 'df' is not defined

- Ref: https://towardsdatascience.com/the-complete-guide-to-linear-regression-in-python-3d3f8f06bf8
- Looking at both coefficients, we have a p-value that is equal to 0. This means that there is a strong correlation between these coefficients and the target (Tip).
- The R² value is equal to 0.45. Therefore, about 45% of the variability of Tips is explained by the Bill amount.

### Analysis
- Analyse the relationship between the variables within the dataset.
- The aim of this experiment is to determine what circumstances maxamises the Tip payable, this is done by minimising the Cost(m,c). Of course the data outlined above doesn't take into account, time, gender, smoker, so the model is not entirely accurate.  

In [ ]:
plt.figure(1)
df.groupby(['Gender', 'Time'])['Tip'].mean().unstack(level=0).plot(kind='bar', figsize=(10,5))
plt.ylabel('Tip Amount')

plt.figure(2)
df.groupby(['Gender', 'Smoker'])['Tip'].mean().unstack(level=0).plot(kind='bar', figsize=(10,5))
plt.ylabel('Tip Amount')

plt.figure(3)
df.groupby(['Party', 'Time'])['Tip'].mean().unstack(level=0).plot(kind='bar', figsize=(10,5))
plt.ylabel('Tip Amount')


plt.show()

- From the first three plots, it would seem that males are better tippers than females and that with the exeption of lunch, that size of the tip increases with the party amount. 

# End

## The following contains some code that was not needed but that I may come back to. 

In [ ]:
B = df["Bill"]
T = df["Tip"]
P = df["Party"]
G = df['Gender']

In [ ]:
#df.Gender = df.Gender.map({'Female':0,'Male':1})
#df['Gender'] = df['Gender'].map(dict(zip(['Female','Male'],[0,1]))
df['Gender'] = (df['Gender'] == 'Female').astype(int)
df.head()


In [ ]:
C = np.polyfit(P, T, 1)
C

In [ ]:
from statsmodels.formula.api import ols

fit = ols('Tip ~ (Bill)', data=df).fit() 

fit.summary()

In [ ]:
m = 0.71182064
c = 1.16913303

cost = lambda m,c: np.sum([(T[i] - m * P[i] - c)**2 for i in range(P.size)])

print("Red line cost with m = %5.2f and c = %5.2f: is %8.2f" % (0.71182064, 1.16913303, cost(0.71182064, 1.16913303)))

In [ ]:
X = df['Party']
y = df['Tip']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
import seaborn as sns

import sklearn.neighbors as nei
# Plot the tips data set with a pair plot.
sns.pairplot(df)

- https://www.accelebrate.com/blog/interpreting-results-from-linear-regression-is-the-data-appropriate